In [3]:
import json 
from pathlib import Path
import pandas as pd
import os


In [4]:
directory = './data/json_files/'

# Lista delle chiavi selezionate per le colonne CSV
selected_keys = ['file_name', 'n_electrons', 'fermi_level_ev', "total_energy_eV","total_energy_eV_-1","total_energy_eV_+1", "IP_ev","EA_ev","band_gap_ev"]

# Lista per tenere traccia di tutti i dizionari da tutti i file JSON
data_list = []

# Scansiona tutti i file nella cartella JSON
for filename in os.listdir(directory):
    if filename.endswith('.json'):
        with open(os.path.join(directory, filename), 'r') as file:
            data = json.load(file)
            # Seleziona solo le chiavi desiderate
            selected_data = {key: data[key] for key in selected_keys}
            data_list.append(selected_data)

# Converti la lista di dizionari in un DataFrame
df = pd.DataFrame(data_list)

df['file_name'] = df['file_name'].str.slice(0, -4)

# Esporta il DataFrame in un file CSV
df.to_csv('target_graphene_dftb.csv', index=False)

In [ ]:
df.head()